<a href="https://colab.research.google.com/github/ayjiang97/Projects/blob/master/Dell_OCE_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Online Cluster Expansion (OCE) Analysis

In [87]:
import numpy as np
import pandas as pd
import seaborn as sns

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Overview

In [89]:
path = '/content/drive/MyDrive/R0682026_v1.1.xlsx'
df = pd.read_excel(path)
df.shape

(868, 5)

In [90]:
# data preview
df.head()

,Identifier,Software Version,Starting X-Brick Count,New X-Brick Count,Theater
0,APM00142902156,4.0.0-64,2,4,Americas CS Theater
1,APM00150502641,4.0.15-15,1,2,Americas CS Theater
2,FNM00161800189,4.0.15-20,1,2,Americas CS Theater
3,FNM00160100226,4.0.15-15,4,8,Americas CS Theater
4,FNM00153600616,4.0.15-20,6,8,Americas CS Theater


In [91]:
# check missing value -- no missing 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Identifier              868 non-null    object
 1   Software Version        868 non-null    object
 2   Starting X-Brick Count  868 non-null    int64 
 3   New X-Brick Count       868 non-null    int64 
 4   Theater                 868 non-null    object
dtypes: int64(2), object(3)
memory usage: 34.0+ KB


In [92]:
# initial software version check 
df['Software Version'].value_counts().sort_index()

4.0.0-64               5
4.0.1-41              25
4.0.1-41_hotfix_1     17
4.0.1-7                9
4.0.10-33             27
4.0.15-15             88
4.0.15-20            141
4.0.15-24            151
4.0.2-65              12
4.0.2-80             208
4.0.2-80_hotfix_1      1
4.0.25-22             17
4.0.25-27             27
4.0.4-28              58
4.0.4-41              82
Name: Software Version, dtype: int64

### Data Cleaning

In [93]:
# remove duplicates
df.drop_duplicates(inplace=True)
df.shape    # (868) -> (811): removed 57 duplicates

(811, 5)

In [94]:
# exclude unrelated verstions
unrelated_version = ['4.0.15-15','4.0.15-20','4.0.15-24','4.0.25-22','4.0.25-27'] 
df = df[~df['Software Version'].isin(unrelated_version)]

In [109]:
# remove Starting X-Brick == 1
df = df[df['Starting X-Brick Count'] != 1]

In [110]:
# double check the software version distribution
df['Software Version'].value_counts().sort_index()

4.0.0-64              2
4.0.1-41             18
4.0.1-41_hotfix_1    12
4.0.1-7               5
4.0.10-33            13
4.0.2-65              3
4.0.2-80             84
4.0.4-28             16
4.0.4-41             38
Name: Software Version, dtype: int64

In [111]:
# the data frame has 195 rows only after filtering
df.shape

(191, 6)

In [112]:
df.head()

,Identifier,Software Version,Starting X-Brick Count,New X-Brick Count,Theater,expansion
0,APM00142902156,4.0.0-64,2,4,Americas CS Theater,"(2, 4)"
11,CKM00151700613,4.0.0-64,2,4,EMEA CS Theater,"(2, 4)"
12,CKM00155101347,4.0.10-33,2,4,APJK CS Theater,"(2, 4)"
13,APM00144709196,4.0.10-33,4,6,Americas CS Theater,"(4, 6)"
18,FNM00160100020,4.0.10-33,2,4,Americas CS Theater,"(2, 4)"


In [113]:
# check software distribution in theater 
df.Theater.value_counts()

Americas CS Theater    157
EMEA CS Theater         27
APJK CS Theater          7
Name: Theater, dtype: int64

EMEA = Europe, the Middle East and Africa

APJK = Asia Pacific, Japan and Korea

### Release History Date

In [114]:
release = [['4.0.0-64','30-Sep-15'],['4.0.1-41','15-Oct-15'],['4.0.2-80','23-Dec-15'],
           ['4.0.4-41','28-Apr-16'],['4.0.10-33','30-Jun-16']]

In [115]:
release_history = pd.DataFrame(release, columns = ['Software Version', 'Release Date'])
release_history

,Software Version,Release Date
0,4.0.0-64,30-Sep-15
1,4.0.1-41,15-Oct-15
2,4.0.2-80,23-Dec-15
3,4.0.4-41,28-Apr-16
4,4.0.10-33,30-Jun-16


### Expasion

In [117]:
# create a new column 'expansion' to store X-Brick info in tuple
df['expansion'] = list(zip(df['Starting X-Brick Count'], df['New X-Brick Count']))
df.sample(5)

,Identifier,Software Version,Starting X-Brick Count,New X-Brick Count,Theater,expansion
459,FNM00160100469,4.0.2-80,2,4,Americas CS Theater,"(2, 4)"
828,FNM00162200352,4.0.4-41,6,8,Americas CS Theater,"(6, 8)"
166,APM00150426258,4.0.1-41_hotfix_1,4,6,Americas CS Theater,"(4, 6)"
453,FNM00151800534,4.0.2-80,2,6,Americas CS Theater,"(2, 6)"
578,FNM00151302259,4.0.2-80,2,4,Americas CS Theater,"(2, 4)"


In [118]:
# find the most popular OCE path
a2 = df.groupby(["expansion"]).size().to_frame(name = 'count').reset_index()
a2 = a2.sort_values(by=['count'], ascending=False)
a2

,expansion,count
0,"(2, 4)",120
3,"(4, 6)",47
5,"(6, 8)",10
4,"(4, 8)",7
1,"(2, 6)",5
2,"(2, 8)",2


The most popular OCE path are (2,4) and (4,6). Due to the high demand of OCE expansion path, Dell could increase the inventory of 4 X-Brick and 6 X-Brick.

In [119]:
# break into detail by adding software version and add release date
a = df.groupby(["Software Version","expansion"]).size().to_frame(name = 'count').reset_index()
a1 = a.sort_values(by=['count'], ascending=False)

add_date = pd.merge(a1, 
                      release_history, 
                      on ='Software Version', 
                      how ='left')
add_date.head()

,Software Version,expansion,count,Release Date
0,4.0.2-80,"(2, 4)",51,23-Dec-15
1,4.0.4-41,"(2, 4)",26,28-Apr-16
2,4.0.2-80,"(4, 6)",18,23-Dec-15
3,4.0.1-41,"(2, 4)",14,15-Oct-15
4,4.0.4-41,"(4, 6)",10,28-Apr-16


The highest expansion frequency is XtremIO 4.0.2-80, which OCE path is 2 X-Brick to 4 X-Brick. The next high frequent expansion version is 4.0.-41 with the same OCE path.

In [120]:
# export
df.to_csv('OCE Analysis_1.csv', index=False)